# Create Entities

A short guide to adding Entities with Minimum Disambiguation Triples(MDTs) to the Golden decentralized knowledge graph using the godel SDK. Please see the [Golden Protocol FAQ Guide](https://www.notion.so/goldenhq/Golden-Protocol-FAQ-78ae2357b9af44aeaa655cb1b1966ee4) and the [Adding Structured Data Guide](https://www.notion.so/goldenhq/Adding-Structured-Data-Guide-ae657337bf4f4e54ae4402df083c76ac) to learn more about Golden, data submission, and rewards.

Note: Attribution and eligibility for testnet points on triple submissions will be assigned by the earliest timestamped transaction.

## Prerequisite

[Authentication](https://docs.golden.xyz/guides/authentication)

[Godel Authentication](https://docs.golden.xyz/godel-python-sdk/authentication)

This guide requires you install Godel's data-tools.

You can do this with `pip install godel` and comes pre-installed if using the godel docker image.

### 1. Connect to Golden Web3 API

Let's connect the python wrapper to the Golden GraphQL API.

Make sure you ran through the prerequisites for this guide and have learned how to authenticate and retrieve your JWT token in Godel.

In [1]:
from godel import GoldenAPI

JWT_TOKEN = ""  # TODO: Enter your JWT token
goldapi = GoldenAPI(jwt_token=JWT_TOKEN)

In [2]:
import pandas as pd

# Test connection with search
search_results = goldapi.entity_search(name="Miles")
search_results_df = pd.DataFrame(search_results["data"]["entityByName"]["nodes"])
search_results_df.head()

,id,name,description,thumbnail,goldenId,pathname
0,f25272da-d6be-4eef-8c06-a33eeafbc5c6,A. D. Miles,American comedy actor and writer,None,1148852,/entity/f25272da-d6be-4eef-8c06-a33eeafbc5c6
1,0b178fa4-eb13-478f-8a8a-234a5bb27112,Miles Teller,American actor,https://golden-media.s3.amazonaws.com/topics/1...,3482005,/entity/0b178fa4-eb13-478f-8a8a-234a5bb27112
2,f32cc7fa-8477-408a-a131-71f01034443f,Mile Škorić,Croatian association football player,https://golden-storage-production.s3.amazonaws...,4847572,/entity/f32cc7fa-8477-408a-a131-71f01034443f
3,37028d18-a2fd-4d9a-9bf9-7e3dfa8790ed,Open Mile (company),Company,https://golden-storage-production.s3.amazonaws...,5368783,/entity/37028d18-a2fd-4d9a-9bf9-7e3dfa8790ed
4,56bd0505-5aa7-4617-a8cd-70f5ac60fb14,MARVEL'S Spider-Man Miles Morales,Marvel's Spider-Man: Miles Morales is a 2020 a...,https://golden-storage-production.s3.amazonaws...,11886359,/entity/56bd0505-5aa7-4617-a8cd-70f5ac60fb14


### 2. Get Predicates and Templates

You can run the code below to get the list of accepted predicates into the knowledge graph and its data types.

In [3]:
import pandas as pd

predicates = {}
for p in goldapi.predicates()["data"]["predicates"]["edges"]:
    p = p["node"]
    predicates[p["name"]] = {"id": p["id"], "objectType": p["objectType"]} 
predicates_df = pd.DataFrame(predicates).transpose()
predicates_df.head()

,id,objectType
Spotify Artist ID,09dec055-52d8-42e6-9fc0-56ecac599a8b,STRING
CEO,0a87e996-34b4-46ba-909a-70ab67b1f811,ENTITY
Email address,0efd0441-1ffc-4e30-8806-e58c434770c8,STRING
YouTube channel,12acb8fe-0573-4ca8-8cc1-180cc6ba3486,ANY_URI
Whitepaper,14fa743c-8161-42e8-a92f-5c29c70e87f8,ANY_URI


In [4]:
templates = {}
for t in goldapi.templates()["data"]["templates"]["edges"]:
    t = t["node"]
    templates[t["entity"]["name"]] = {"id": t["id"], "entityId": t["entityId"], "entityDescription": t["entity"]["description"]} 
templates_df = pd.DataFrame(templates).transpose()
templates_df

,id,entityId,entityDescription
Person,0dea0f27-ce0c-4b4f-8ddb-5ff6adb57e12,0c4e6054-5fd8-48a8-817c-f6611278f755,None
Company,9553f193-a46a-4b46-9c0f-5287289644a6,0a9fcc89-e14b-47af-85c3-8465ca607c29,None


### 4. Create Entity

#### Source Data

You will need to input the source data of the entity you would like to submit.

In [5]:
name = "John Doe"
is_a = templates_df["entityId"]["Person"]  # Person Entity ID 
ceo_of = "36b56cea-db86-482f-8b63-5b4e75b3b968"  # "Huel" Entity ID
email_address = "john.doe@example.com"
citation_urls = ["https://golden.com/wiki/johndoe"]

#### Create Entity Input

We're going to use `CreateEntityInput`, which requires Minimum Disambiguated Triples (MDTs), to submit the new entity.

MDTs are the required triples you must submit along with the entity you want to create. This helps us with disambiguation, deduplication, and arbitrary entity submissions with zero triples.

First, view the `StatementInputRecordInput` object schema.

In [6]:
from godel.schema import CreateEntityInput, StatementInputRecordInput

print(repr(CreateEntityInput))
StatementInputRecordInput

input CreateEntityInput {
  clientMutationId: String
  statements: [StatementInputRecordInput]
}


input StatementInputRecordInput {
  predicateId: UUID!
  objectValue: String
  objectEntityId: UUID
  citationUrls: [String]
  qualifiers: [QualifierInputRecordInput]
}

Create your triples with `StatementInputRecordInput`.

You'll notice that the "CEO of" statement is commented out. Without this, even if you have the email address statement, you will not be able to submit the entity since it does not fulfill the MDTs required.

Remove the comment-out of the "CEO of" statement to succesfully submit the entity.

In [7]:
# Create triples inputs
statements = []

# Add Template
statements.append(
    StatementInputRecordInput(
        predicate_id=predicates["Is a"]["id"],
        object_entity_id=is_a,
        citation_urls=citation_urls,
        qualifiers=[],
    )
)

# Add Name
statements.append(
    StatementInputRecordInput(
        predicate_id=predicates["Name"]["id"],
        object_value=name,
        citation_urls=citation_urls,
        qualifiers=[],
    )
)

# Add Email Address
statements.append(
    StatementInputRecordInput(
        predicate_id=predicates["Email address"]["id"],
        object_value=email_address,
        citation_urls=citation_urls,
        qualifiers=[],
    )
)

# Add CEO of
# statements.append(
#     StatementInputRecordInput(
#         predicate_id=predicates["CEO of"]["id"],
#         object_entity_id=ceo_of,
#         citation_urls=citation_urls,
#         qualifiers=[],
#     )
# )

Now that you have the statements, you can create your `CreateEntityInput`.

In [8]:
# Create Entity Input
create_entity_input = CreateEntityInput(
    statements=statements
)
create_entity_input.__to_json_value__()

{'statements': [{'predicateId': '94a8d215-ce32-4379-b18e-2aebf0794882',
   'objectEntityId': '0c4e6054-5fd8-48a8-817c-f6611278f755',
   'citationUrls': ['https://golden.com/wiki/johndoe'],
   'qualifiers': []},
  {'predicateId': 'a27218b8-6a4d-47bb-95b6-5a55334fac1c',
   'objectValue': 'John Doe',
   'citationUrls': ['https://golden.com/wiki/johndoe'],
   'qualifiers': []},
  {'predicateId': '0efd0441-1ffc-4e30-8806-e58c434770c8',
   'objectValue': 'john.doe@example.com',
   'citationUrls': ['https://golden.com/wiki/johndoe'],
   'qualifiers': []}]}

### WARNING: Running the code below may charge gas fees and stake testnet points with your wallet. You may lose testnet points by submitting incorrect data.

#### Submit entity data to protocol

In [9]:
data = goldapi.create_entity(create_entity_input=create_entity_input)
data

GraphQL query failed with 1 errors


{'errors': [{'errcode': '22023',
   'extensions': {'messages': [], 'exception': {'errcode': '22023'}},
   'message': 'All required statements must be provided:\n- entity type 0c4e6054-5fd8-48a8-817c-f6611278f755 must have a valid statement from at least one of these predicates: 7f15d788-5df1-4ff3-a5e5-4c9e8e2c57af, 8c4d6279-199f-4e46-9ef7-8702bad1e152, 42cb158b-e836-45ed-9b56-034668b8f05a, 3104de39-071c-47b8-86b4-d62ccc4a4fa6, 2f30a94e-cd5e-496f-bec8-01bfb01da128, e4f94b98-c56a-4bd2-a9fd-5fd11603e7e8',
   'locations': [{'line': 2, 'column': 1}],
   'path': ['createEntity'],
   'stack': ['error: All required statements must be provided:',
    '- entity type 0c4e6054-5fd8-48a8-817c-f6611278f755 must have a valid statement from at least one of these predicates: 7f15d788-5df1-4ff3-a5e5-4c9e8e2c57af, 8c4d6279-199f-4e46-9ef7-8702bad1e152, 42cb158b-e836-45ed-9b56-034668b8f05a, 3104de39-071c-47b8-86b4-d62ccc4a4fa6, 2f30a94e-cd5e-496f-bec8-01bfb01da128, e4f94b98-c56a-4bd2-a9fd-5fd11603e7e8',
  

In [20]:
# Below is a dataframe of your new entity once submitted with the required MDTs. 
created_data_df = pd.DataFrame([data["data"]["createEntity"]["entity"]])
created_data_df

,__typename,id,pathname,name,description,thumbnail,goldenId,isA,statementsBySubjectId
0,Entity,5c3465d5-4a9a-46a3-a9cc-eeb5113ebc40,/entity/5c3465d5-4a9a-46a3-a9cc-eeb5113ebc40,John Doe,None,None,None,{'nodes': [{'id': '0c4e6054-5fd8-48a8-817c-f66...,"{'nodes': [{'__typename': 'Statement', 'id': '..."


### 6. View Data on dApp

In [21]:
created_entity_id = data["data"]["createEntity"]["entity"]["id"]
created_entity_id
link = f"https://dapp.golden.xyz/entity/{created_entity_id}"
link

'https://dapp.golden.xyz/entity/5c3465d5-4a9a-46a3-a9cc-eeb5113ebc40'